**# Importanto o Dataset e verificando o conteúdo**

In [1]:
import sys
import os
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') #carrego stopwrds para filtragem NLP

# Adiciona o caminho da pasta onde o arquivo 'dataset.py' está localizado
# Isso pula a necessidade de mencionar a pasta com hífen no import
caminho_raw = os.path.abspath(os.path.join('..', 'data', 'raw'))
if caminho_raw not in sys.path:
    sys.path.append(caminho_raw)

from dataset import carregar_dados

df = carregar_dados()
print(df.head())

[nltk_data] Downloading package stopwords to /home/rafael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/rafael/Desktop/Portifolio/Intelligent-Assistant-for-Data-and-Document-Analysis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/rafael/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Baixando arquivo do Kaggle e movendo o dataset para pasta do projeto 


100%|██████████| 980M/980M [03:53<00:00, 4.41MB/s] 

Extracting files...


Arquivo salvo em: /home/rafael/Desktop/Portifolio/Intelligent-Assistant-for-Data-and-Document-Analysis/data/raw/github_issues.csv
                                           issue_url  \
0  "https://github.com/zhangyuanwei/node-images/i...   
1     "https://github.com/Microsoft/pxt/issues/2543"   
2  "https://github.com/MatisiekPL/Czekolada/issue...   
3  "https://github.com/MatisiekPL/Czekolada/issue...   
4  "https://github.com/MatisiekPL/Czekolada/issue...   

                                         issue_title  \
0  can't load the addon. issue to: https://github...   
1  hcl accessibility a11yblocking a11ymas mas4.2....   
2  issue 1265: issue 1264: issue 1261: issue 1260...   
3  issue 1266: issue 1263: issue 1262: issue 1259...   
4  issue 1288: issue 1285: issue 1284: issue 1281...   

                                                body  
0  can't load the addon. issue to: https://github...  
1  user experience: user who depends on screen re...  
2  ┆attachments: <a href= https

**Processamento e limpeza dos dados**


In [2]:
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'raw')))
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'processed')))


from dataset import verifica_vazios, verifica_frequentes
from data_processing import clean_text

df["clean_title"] = df["issue_title"].apply(clean_text)
df["clean_body"] = df["body"].apply(clean_text)


media_palavras_url = df["issue_url"].str.split().str.len().mean()
media_palavras_title = df["clean_title"].str.split().str.len().mean()
media_palavras_body = df["clean_body"].str.split().str.len().mean()

print(f"Média de palavras em url: {media_palavras_url}, em title: {media_palavras_title}, em body: {media_palavras_body}")

url_vazio = verifica_vazios(df,"issue_url")
titles_vazio = verifica_vazios(df,"clean_title")
body_vazio = verifica_vazios(df,"clean_body")

palavras_frequentes_titles = verifica_frequentes(df,"clean_title", 20)
print(f"Palavras mais frequentes em titles: {palavras_frequentes_titles}")
palavras_frequentes_body = verifica_frequentes(df,"clean_body", 20)
print(f"Palavras mais frequentes em body: {palavras_frequentes_body}")



Média de palavras em url: 1.0, em title: 5.459, em body: 28.4455
Resultados para 'issue_url':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 0
Resultados para 'clean_title':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 99
Resultados para 'clean_body':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 11
Palavras mais frequentes em titles: [('issu', 352), ('add', 151), ('bug', 137), ('found', 79), ('error', 49), ('test', 43), ('use', 39), ('api', 39), ('help', 34), ('fix', 34), ('new', 33), ('need', 26), ('log', 24), ('run', 23), ('list', 22), ('get', 20), ('set', 19), ('line', 17), ('typo', 17), ('task', 16)]
Palavras mais frequentes em body: [('close', 1244), ('add', 1102), ('issues', 927), ('columns', 714), ('list', 585), ('comment', 558), ('column', 542), ('trello', 529), ('update', 396), ('default', 387), ('settings', 378), ('via', 376), ('sync', 373), ('move', 369), ('custom', 367), ('board', 358), ('card', 357), ('attachments', 355), ('cards', 353), ('matisiekpl', 352)]



   **Análise descritiva dos dados processados**

- Média de palavras para entender quantos tokens irão ser processados pela LLM
    - Quantidade de células vazias em todas as colunas
        - não teve resultado de células vazias, todas preenchidas
    - Títulos - tamanhos médio de 5,4 palavras
    - Body - Aproximadamente 28 palavras
- Ideal para:
    - Embeddings
    - Chunking Leve
    - RAG Eficiente(baixo custo e boa semântica)

**Criação de Texto final**

In [ ]:
from sentence_transformers import SentenceTransformer

#criação de nova coluna para texto final que será direcionado ao embedding
df["final_text"] = (
    "Title: " + df["clean_title"] +
    ". Body: " + df["clean_body"]
)

#carregando o modelo que será usado, modelo rápido e leve para projeto OBS: Uso da CPU pois GPU esta ultrapassada para o modelo
model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")
df[["clean_title", "clean_body", "final_text"]].head()

,clean_title,clean_body,final_text
0,load addon issue error lib libc version glibc ...,load addon issue error lib libc version glibc ...,Title: load addon issue error lib libc version...
1,hcl accessibility yblocking ymas mas hcl makec...,user experience user depends screen reader get...,Title: hcl accessibility yblocking ymas mas hc...
2,issue issue issue issue issue issue issue issu...,attachments github com matisiekpl czekolada is...,Title: issue issue issue issue issue issue iss...
3,issue issue issue issue issue issue issue issu...,gitlo github trello board linked update issue ...,Title: issue issue issue issue issue issue iss...
4,issue issue issue issue issue issue issue issu...,attachments github com matisiekpl czekolada is...,Title: issue issue issue issue issue issue iss...


**Embedding**

In [ ]:
import numpy as np

#listando o texto final em variavel para ser codificada
texts = df["final_text"].tolist()

#realização do embedding pelo modelo escolhido
embeddings = model.encode( 
    texts,
    show_progress_bar= True
)

#criação do array em np 
embeddings = np.array(embeddings)
print(embeddings.shape)

#persistindo os dados em formato npy e csv para nao necessitar de conversao novamente
np.save("embeddings.npy",embeddings)
df.to_csv("issue_processed.csv", index=False)


Batches: 100%|██████████| 63/63 [00:17<00:00,  3.64it/s]


(2000, 384)
